In [26]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pyodbc 
from datetime import datetime
import math
import time

In [ ]:
from scrapping_functions import discover_all_categories

In [17]:
def discover_popular_products():
    """
    Scrappe la page "populaires" (tri par défaut) de Vanden Borre.
    
    Cette fonction scrappe le HTML de la page catégorie pour trouver
    les produits listés.
    
    Retourne:
        Une liste de dictionnaires. Ex:
        [
            {'sku': '7819145', 'url': 'https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black'},
            {'sku': '7683081', 'url': 'https://www.vandenborre.be/fr/casque/sony-wh-1000xm6-noir'}
        ]
    """
    URL_DECOUVERTE = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque"
    BASE_URL = "https://www.vandenborre.be"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    }

    print(f"Scraping de la page de découverte : {URL_DECOUVERTE}...")
    
    try:
        session = requests.Session()
        session.headers.update(headers)
        response = session.get(URL_DECOUVERTE, timeout=15)
        response.raise_for_status() # Stoppe si erreur 403, 404, etc.
        
        soup = BeautifulSoup(response.content, 'html.parser')

        # --- Extraction HTML (la bonne méthode pour VDB) ---
        # On cherche tous les blocs qui contiennent un produit
        product_containers = soup.find_all('div', {'class': 'js-product-container'})
        
        if not product_containers:
            print("⚠️ Aucun conteneur produit trouvé avec 'div.js-product-container'.")
            return []

        print(f"   -> {len(product_containers)} produits trouvés sur la page.")
        
        # --- Transformation ---
        product_list = []

        for container in product_containers:
            sku = container.get('data-productid')
            if not sku:
                continue # Passe au suivant si pas de SKU

            # Trouver le lien (qui contient l'URL)
            link_tag = container.find('a', {'class': 'js-product-click'})
            if not link_tag or not link_tag.get('href'):
                continue # Passe au suivant si pas d'URL

            # Reconstruction de l'URL
            relative_url = link_tag['href']
            product_url = ""
            if relative_url.startswith('//'):
                product_url = f"https:{relative_url}"
            elif relative_url.startswith('/'):
                product_url = f"{BASE_URL}{relative_url}"
            else:
                product_url = relative_url # Au cas où
            
            # Ajoute le produit à la liste
            product_list.append({
                "sku": sku,
                "url": product_url
            })
                 
        return product_list

    except requests.exceptions.HTTPError as err:
        print(f"\n--- ❌ ERREUR HTTP : {err} ---")
        return []
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")
        return []

In [18]:

products_to_scrape = discover_popular_products( )

if products_to_scrape:
    print(f"\n--- SUCCÈS : {len(products_to_scrape)} produits à scraper ont été identifiés ---")
   
    print(json.dumps(products_to_scrape, indent=2, ensure_ascii=False))
else:
    print("\n--- ÉCHEC : Le scraping n'a retourné aucun produit.  ---")

Scraping de la page de découverte : https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque...
   -> 29 produits trouvés sur la page.

--- SUCCÈS : 24 produits à scraper ont été identifiés ---
[
  {
    "sku": "7819145",
    "url": "https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black"
  },
  {
    "sku": "7683081",
    "url": "https://www.vandenborre.be/fr/casque/sony-wh-1000xm6-noir"
  },
  {
    "sku": "7762429",
    "url": "https://www.vandenborre.be/fr/casque/jbl-live-770nc-noir"
  },
  {
    "sku": "7825773",
    "url": "https://www.vandenborre.be/fr/casque/jbl-tune-520-bt-black"
  },
  {
    "sku": "7820852",
    "url": "https://www.vandenborre.be/fr/casque/jbl-tune-670nc-black"
  },
  {
    "sku": "8350248",
    "url": "https://www.vandenborre.be/fr/casque/jbl-t500-black"
  },
  {
    "sku": "7755562",
    "url": "https://www.vandenborre.be/fr/casque/marshall-major-v-black"
  },
  {
    "sku": "8190895",
    "url": "https://www.vandenborre.be/fr/casque/sony-wh-1000xm4-no

In [9]:


# --- Configuration SQL Server (Authentification Windows) ---
DB_CONFIG = {
    'server': 'LAPTOP-VT8FTHG2\DATAENGINEER', 
    'database': 'Projet_Market_Staging',
    'driver': '{ODBC Driver 17 for SQL Server}',
    'connection_string': (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=LAPTOP-VT8FTHG2\DATAENGINEER;" # Remplace ceci
        "DATABASE=Projet_Market_Staging;"
        "Trusted_Connection=yes;"
    )
}



In [11]:
DB_CONFIG = {
    'server': r'LAPTOP-VT8FTHG2\DATAENGINEER', 
    'database': 'Projet_Market_Staging',
    'driver': '{ODBC Driver 17 for SQL Server}' 
}

# Construit la chaîne de connexion pour l'authentification Windows
conn_str = (
    f"DRIVER={DB_CONFIG['driver']};"
    f"SERVER={DB_CONFIG['server']};"
    f"DATABASE={DB_CONFIG['database']};"
    "Trusted_Connection=yes;" # La ligne clé
)

In [12]:
if products_to_scrape:
    print(f"\n--- Connexion à SQL Server : {DB_CONFIG['server']}...")
    conn = None
    cursor = None
    try:
        conn = pyodbc.connect(conn_str, autocommit=False)
        cursor = conn.cursor()
        print("   -> ✅ Connecté à SQL Server avec succès.")

        insert_count = 0
        for product in products_to_scrape:
            sku = product['sku']
            url = product['url']
            
            # Vérifie si le produit n'est pas déjà en file d'attente
            cursor.execute("SELECT 1 FROM Staging_Scraping_Queue WHERE ProductID_SKU = ? AND Status = 'pending'", (sku))
            if cursor.fetchone() is None:
                # S'il n'y est pas, on l'insère
                cursor.execute(
                    "INSERT INTO Staging_Scraping_Queue (ProductID_SKU, ProductURL, Status, DiscoveredAt) VALUES (?, ?, 'pending', GETDATE())",
                    (sku, url)
                )
                insert_count += 1
        
        conn.commit() # Valide les insertions
        print(f"   -> ✅ {insert_count} nouveaux produits insérés dans Staging_Scraping_Queue.")
        
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"\n--- ❌ ERREUR SQL Server : {sqlstate} ---")
        print("   Vérifie tes 'DB_CONFIG': nom du serveur, nom de la base, et que le driver ODBC est installé.")
        print(f"   Chaîne de connexion tentée : {conn_str}")
        if 'conn' in locals() and conn: conn.rollback()
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")
        if 'conn' in locals() and conn: conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
            print("Connexion SQL Server fermée.")
else:
    print("\n--- ÉCHEC : Aucun produit n'a été scrapé, rien à insérer dans la base de données. ---")


--- Connexion à SQL Server : LAPTOP-VT8FTHG2\DATAENGINEER...
   -> ✅ Connecté à SQL Server avec succès.
   -> ✅ 24 nouveaux produits insérés dans Staging_Scraping_Queue.
Connexion SQL Server fermée.


In [22]:
def discover_all_products(category_url):
    """
    Scrappe TOUTES les pages d'une catégorie donnée de Vanden Borre
    en calculant dynamiquement le nombre total de pages.
    
    Elle utilise la méthode de scraping HTML (div.js-product-container)
    qui a fonctionné.
    
    Args:
        category_url (str): L'URL de base de la catégorie 
                            (ex: "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque")
        
    Retourne:
        Une liste de dictionnaires (ta "variable"). Ex:
        [
            {'sku': '7819145', 'url': 'https://www.vandenborre.be/.../jbl-tune-770nc-black'},
            ...
            (Tous les produits de toutes les pages)
            ...
        ]
    """
    BASE_URL = "https://www.vandenborre.be"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    
    all_products_found = []
    session = requests.Session()
    session.headers.update(headers)
    
    # --- 1. Découverte du nombre de pages ---
    print(f"--- Étape 0 : Calcul du nombre de pages ---")
    print(f"Scraping de la Page 1 pour les métadonnées : {category_url}")
    
    try:
        # On utilise le tri par défaut (populaires, sorting=1)
        url_page_1 = category_url
        response = session.get(url_page_1, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Trouver le nombre total de produits (basé sur le HTML fourni)
        count_text = soup.find('span', {'class': 'js-filter-counttotal'}).text # ex: "174"
        total_count = int(re.sub(r'[^\d]', '', count_text)) # Nettoyage au cas où
        
        # Trouver le nombre par page
        count_per_page_select = soup.find('select', {'name': 'COUNTPERPAGE'})
        count_per_page = int(count_per_page_select.find('option', {'selected': True}).get('value', 24))
        
        if total_count == 0:
            raise Exception("Le 'totalCount' est 0. Impossible de continuer.")
            
        # math.ceil() arrondit au nombre supérieur (ex: 174 / 24 = 7.25 -> 8 pages)
        total_pages = math.ceil(total_count / count_per_page)
        
        print(f"   -> ✅ Découvert : {total_count} produits sur {total_pages} pages ({count_per_page} par page).")
        
    except Exception as e:
        print(f"\n--- ❌ ERREUR lors de la découverte des métadonnées : {e} ---")
        return [] # Arrête tout

    # --- 2. Boucle de Scraping de toutes les pages ---
    print(f"\n--- Lancement du Scraping des {total_pages} pages ---")
    
    try:
        for page_num in range(1, total_pages + 1):
            print(f"Scraping de la Page {page_num}/{total_pages}...")
            
            # On ré-utilise le 'soup' de la page 1 qu'on a déjà
            if page_num > 1:
                url_to_scrape = f"{category_url}?page={page_num}&sorting=1"
                response = session.get(url_to_scrape, timeout=15)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
            
            product_containers = soup.find_all('div', {'class': 'js-product-container'})
            
            if not product_containers:
                print(f"   -> ⚠️ Page {page_num} vide. On continue...")
                continue

            products_on_this_page = 0
            for container in product_containers:
                sku = container.get('data-productid')
                if not sku: continue 

                link_tag = container.find('a', {'class': 'js-product-click'})
                if not link_tag or not link_tag.get('href'): continue 

                relative_url = link_tag['href']
                product_url = ""
                if relative_url.startswith('//'):
                    product_url = f"https:{relative_url}"
                elif relative_url.startswith('/'):
                    product_url = f"{BASE_URL}{relative_url}"
                else:
                    product_url = relative_url
                
                all_products_found.append({"sku": sku, "url": product_url})
                products_on_this_page += 1
            
            print(f"   -> {products_on_this_page} produits extraits de cette page.")
            
            if page_num < total_pages:
                 print("   -> Pause de 2 secondes...")
                 time.sleep(2) # Pause de politesse
        
        print(f"\n--- Scraping terminé. {len(all_products_found)} produits découverts au total. ---")
        return all_products_found

    except requests.exceptions.HTTPError as err:
        print(f"\n--- ❌ ERREUR HTTP pendant la boucle : {err} ---")
        return all_products_found 
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE pendant la boucle : {e} ---")
        return all_products_found

In [27]:
URL_DECOUVERTE = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque"
products_to_scrape = discover_all_products(URL_DECOUVERTE)

if products_to_scrape:
    print(f"\n--- SUCCÈS : {len(products_to_scrape)} produits à scraper ont été identifiés ---")
   
    print(json.dumps(products_to_scrape, indent=2, ensure_ascii=False))
else:
    print("\n--- ÉCHEC : Le scraping n'a retourné aucun produit.  ---")

--- Étape 0 : Calcul du nombre de pages ---
Scraping de la Page 1 pour les métadonnées : https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque
   -> ✅ Découvert : 174 produits sur 8 pages (24 par page).

--- Lancement du Scraping des 8 pages ---
Scraping de la Page 1/8...
   -> 24 produits extraits de cette page.
   -> Pause de 2 secondes...
Scraping de la Page 2/8...
   -> 24 produits extraits de cette page.
   -> Pause de 2 secondes...
Scraping de la Page 3/8...
   -> 24 produits extraits de cette page.
   -> Pause de 2 secondes...
Scraping de la Page 4/8...
   -> 24 produits extraits de cette page.
   -> Pause de 2 secondes...
Scraping de la Page 5/8...
   -> 24 produits extraits de cette page.
   -> Pause de 2 secondes...
Scraping de la Page 6/8...
   -> 24 produits extraits de cette page.
   -> Pause de 2 secondes...
Scraping de la Page 7/8...
   -> 24 produits extraits de cette page.
   -> Pause de 2 secondes...
Scraping de la Page 8/8...
   -> 24 produits extraits de cette p

In [28]:
len(products_to_scrape)

192